###Perform transformation and UPSERT-MERGE###

In [0]:
%sql
select * from datamodeling.bronze.bronze_table

###Applying upper case for customer_name and adding processDate column###


####selecting only newly inserted records as well ####

In [0]:
spark.sql("""
select *,
upper(customer_name) as Customer_Name_Upper, date(current_timestamp()) as processDate from datamodeling.bronze.bronze_table""").createOrReplaceTempView("silver_source")

In [0]:
%sql
select * from silver_source

###Python_code for creating destination table ###

In [0]:
if spark.catalog.tableExists('datamodeling.silver.silver_table'):
  pass
else:
  spark.sql("""
    CREATE table IF NOT EXISTS datamodeling.silver.silver_table
    as
    select * from silver_source
    """)

###Creating Merge Statement using pyspark###

In [0]:
if spark.catalog.tableExists('datamodeling.silver.silver_table'):
  src = spark.sql("""
    select * from silver_source
    """)
  
else:
  spark.sql("""
    CREATE table IF NOT EXISTS datamodeling.silver.silver_table
    as
    select * from silver_source
    """)

###Merge Using SQL###

In [0]:
%sql
CREATE table IF NOT EXISTS datamodeling.silver.silver_table
    as
    select * from silver_source

In [0]:
%sql
merge into datamodeling.silver.silver_table
using silver_source
on datamodeling.silver.silver_table.order_id = silver_source.order_id
when matched then update set *
when not matched then insert *

In [0]:
%sql
select * from datamodeling.silver.silver_table